<a href="https://colab.research.google.com/github/sriharikrishna/siamcse23/blob/main/tapenade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tapenade mini benchmark

In this notebook we differentiate the Rosenbrock function with Tapenade in reverse mode.

We use [Tapenade](`http://tapenade.inria.fr:8080`) and C for this. Apologies if the syntax highlighting for C is not perfect.

Download Tapenade and unpack it

In [ ]:
%%script bash
if [ ! -f ./tapenade_3.16/bin/tapenade ]; then
    wget https://tapenade.gitlabpages.inria.fr/tapenade/distrib/tapenade_3.16.tar
    tar -xvf tapenade_3.16.tar
fi



## Primal Function

\begin{equation}
F(x) = \sum_{i=0}^{N-1}100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2.
\end{equation}

In [ ]:
%%writefile rosenbrock.c
double rosen(double *x, const int n)
{
    /*
     * Input: x array of values
     * Output: Result of Rosenbrock's banana function
     */
    double sum = 0.0;
    for (int i = 0; i < n - 1; i ++) {
        double c1 = x[i + 1] - x[i] * x[i];
        double c2 = 1.0 - x[i];
        sum += 100.0 * c1 * c1 + c2 * c2;
    }
    return sum;
}

## Primal Driver

This driver takes the problem sizes from the terminal input, sets up the input and calls the function. You probably do not need to modify the driver.



In [ ]:
%%writefile driver.c
#include <stdio.h>
#include <stdlib.h>

double rosen(double *x, const int n);

int main(int argc, char** argv) {
  /*
  Problem setup. Get input/output size, allocate memory, etc.
  */
  if(argc < 2) {
    printf("Usage: driver <num_inputs>");
    return -1;
  }
  int n = atoi(argv[1]);
  printf("===============================================\n");
  printf("Running Rosenbrock function with %d inputs.\n",n);
  double* x = (double*)malloc(n*sizeof(double));
  for(int i=0; i<n; i++) {
    x[i] = 0.5;
  }
  printf("The result of Rosenbrock's is %lf \n", rosen(x, n));
  return 0;
}

In [121]:
%%script bash
gcc rosenbrock.c driver.c -o rosen

In [ ]:
%%bash
./rosen 10

## Reverse Mode AD using Tapenade

We will now apply Tapenade in reverse mode to the function above, again choosing `x` as independent input and `rosen` as dependent output. 

In [ ]:
%%bash
/content/tapenade_3.16/bin/tapenade -b -head "rosen(rosen)/(x)" rosenbrock.c

We can now take a look at the output.

In [108]:
%%bash
cat rosenbrock_b.c

/*        Generated by TAPENADE     (INRIA, Ecuador team)
    Tapenade 3.16 (develop) - 21 Feb 2023 17:28
*/
#include <adStack.h>

/*
  Differentiation of rosen in reverse (adjoint) mode:
   gradient     of useful results: rosen
   with respect to varying inputs: *x
   RW status of diff variables: x:(loc) *x:out rosen:in-killed
   Plus diff mem management of: x:in
*/
void rosen_b(double *x, double *xb, const int n, double rosenb) {
    /*
     * Input: x array of values
     * Output: Result of Rosenbrock's banana function
     */
    double sum = 0.0;
    double sumb = 0.0;
    for (int i = 0; i < n-1; ++i) {
        double c1 = x[i + 1] - x[i]*x[i];
        double c2 = 1.0 - x[i];
        double c1b;
        double c2b;
        pushReal8(c2);
        pushReal8(c1);
    }
    sumb = rosenb;
    *xb = 0.0;
    for (int i = n-2; i > -1; --i) {
        double c1;
        double c1b = 0.0;
        double c2;
        double c2b = 0.0;
        popReal8(&c1);
        popReal8(&c2);
        c

A file called `rosenbrock_b.msg` will contain warnings and error messages, if any.

In [ ]:
%%bash
cat rosenbrock_b.msg

## Driver

This driver takes the problem sizes from the terminal input, sets up the input vector, and calls the function and its derivatives. You probably do not need to modify the driver.

In [ ]:
%%writefile driver_deriv.c

#include <stdio.h>
#include <stdlib.h>

double rosen(double *x, const int n);
void rosen_b(double *x, double *xb, const int n, double rosenb);

int main(int argc, char** argv) {
  /*
  Problem setup. Get input/output size, allocate memory, etc.
  */
  if(argc < 2) {
    printf("Usage: driver <num_inputs>");
    return -1;
  }
  int n = atoi(argv[1]);
  printf("===============================================\n");
  printf("Running Rosenbrock function with %d inputs.\n",n);
  double* x = (double*)malloc(n*sizeof(double));
  for(int i=0; i<n; i++) {
    x[i] = 0.5;
  }

  double *xb = (double*)malloc(n*sizeof(double));
  double rosenb = 1.0;

  /*
  Run the primal function
  */
  double res_primal = rosen(x, n);
  printf("The result of Rosenbrock's is %lf \n", res_primal);

  /*
  Run the reverse-mode
  */
  printf("\nComputing the Derivatives:\n");
  rosen_b(x, xb, n, rosenb);
  
  printf("The derivatives are:" );
  for(int i=0; i<n; i++){
    printf(" %lf ", xb[i]);
  }
  
  free(x); free(xb); 
  return 0;
}

## Compile

Compile our code.

In [127]:
%%script bash

## if the AD runtime library is not yet built, compile it now.
if [ ! -f ADFirstAidKit/adStack.o ]; then
    (cd ./tapenade_3.16/ADFirstAidKit && gcc -c -O3 -w adStack.c adContext.c && ar -crs libAD.a *.o)
fi
gcc -O3 -I./tapenade_3.16/ADFirstAidKit -L./tapenade_3.16/ADFirstAidKit rosenbrock.c rosenbrock_b.c driver_deriv.c -lAD -o rosen_deriv

## Run

Here we run the driver for computing the derivatives.

In [ ]:
%%script bash

./rosen_deriv 10

# Exercise


1.   Change the function `fun` to something you actually want to compute (make sure the function header stays the same if you want to leave the rest of the notebook as is). 

